In [ ]:
import numpy as np
from rsciio import bruker, emd, digitalmicrograph
from jupyterlab_h5web import H5Web
import h5py
from matplotlib import pyplot as plt
import xraydb
from ase.data import chemical_symbols

In [ ]:
src = "/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy"
fnms = [("pdi", "InGaN_nanowires_spectra.edaxh5"),
        ("ikz", "AlGaO.nxs"),
        ("ikz", "GeSi.nxs"),
        ("ikz", "GeSn_13.nxs"),
        ("ikz", "VInP_108_L2.h5"),
        ("fhi", "CG71113 1513 HAADF-DF4-DF2-BF 1.2 Mx STEM.emd"),
        ("adrien", "1613_Si_HAADF_610_kx.emd"),
        ("bruker", "pynx/46_ES-LP_L1_brg.bcf"),
        ("emd", "pynx/1613_Si_HAADF_610_kx.emd"),
        ("digitalmicrograph", "pynx/EELS_map_2_ROI_1_location_4.dm3"),
        ("oxfordinstruments", "pynx/H5OINA_examples_Specimen_1_Map_EDS_+_EBSD_Map_Data_2.h5oina")]
# pyUSID, HSMA
case = 5  # 5 # len(fnms) - 1  # len(fnms) - 1
fnm = f"{src}/{fnms[case][0]}/{fnms[case][1]}"
print(fnm)

In [ ]:
H5Web(fnm)

In [ ]:
# resulting NeXus artifact
H5Web(f"debug.{fnms[case][1]}.nxs")

In [ ]:
objs = emd.file_reader(fnm)
print(len(objs))
for obj in objs:
    if not isinstance(obj, dict):
        raise ValueError("No dict!")
    print(obj.keys())
    for key, val in obj.items():
        print(f"{key}, {np.shape(val)}")
    print(obj["metadata"])
    # print(obj["original_metadata"])
# print(f"{type(objs[0])}")
# print(objs[0].keys())